# Выгрузка данных по нескольким ЦА

## Инициализация

Первым шагом любого ноутбука для построения отчетов необходимо загрузить библиотеки, которые нам помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()


Loading BokehJS ...

# Формируем задания

## Общие параметры для заданий

Для похожих отчетов часто используются одни и те же параметры, удобно их вынести в отдельные переменные, и затем использовать в коде. Если вдруг понадобится какой-то параметр изменить, его нужно поменять в одном месте.

In [2]:
# Задаем параметры

facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2021-03-01'
date_to = '2021-03-31'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# Важно!
# Если в демографической переменной используется возраст, то переменной ages должно быть присвоено значение None
# Если в демографической переменной используется численность населения города, то переменной population должно быть присвоено значение None

# задаем Население, например, Россия 0+
population = rcats.get_population('Russia0+')

# присваиваем переменной ages значение None
ages = None

# Задаем список статистик для расчета
statistics=["UnwReach"]

# Задаем медиа фильтр
media_filter = None

#Создадим словарь для демо переменных

groups = {'12+': 'AGE_GROUPS IN(1, 2, 3, 4, 5, 6, 7)',
          '12-24':'AGE_GROUPS IN(1, 2)', 
          '25-34':'AGE_GROUPS = 3', 
          '35-44':'AGE_GROUPS = 4', 
          '45-54':'AGE_GROUPS = 5', 
          '55+':'AGE_GROUPS IN(6, 7)', 
          'Муж.12+':'SEX = 1', 
          'Жен.12+':'SEX = 2'}

# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-07-02 10:32:36
Тип установки: mobile
Начало периода: 2021-03-01
Конец периода: 2021-03-31
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3, 4]
Возрастные группы: None
Объектов в media-каталоге: 5613


# Расчет Unweighted Reach для нескольких ЦА с разбивкой и без разбивки по Usetype

In [7]:
%%time

#Посчитаем данные в циклах
tasks = []
for agegroup, logical in groups.items():
    
    # Формируем задание на расчет ЦА без рабивки по Usetype
    project_name = f"Desktop&Mobile {agegroup}"
    demo_filter = f"{logical}"
    structure = {
        "usetype": False,
        "date": "month"
    }
    print(f"Отправляем на расчет: {project_name}")

    # Формируем из заданных параметров заданние для Responsum в формате JSON для каждого из этапов цикла
    task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
    
    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name
    tsk['task'] = rtask.send_audience_task(task_json) 
    tasks.append(tsk)

    
    # Формируем задание на расчет ЦА c рабивкой по Usetype
    project_name = f"Desktop и Mobile отдельно {agegroup}"
    demo_filter = f"{logical}"
    structure = {
        "usetype": True,
        "date": "month"
    }
    print(f"Отправляем на расчет: {project_name}")

    # Формируем из заданных параметров заданние для Responsum в формате JSON для каждого из этапов цикла
    task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
    
    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name
    tsk['task'] = rtask.send_audience_task(task_json)
    tasks.append(tsk)
    
# Ждем выполнения
tsks = rtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

#Получаем результат
results = []
for t in tasks:
    tsk = t['task']
    df_result = rtask.result2table(rtask.get_result(tsk), project_name = t['project_name'])
    results.append(df_result)
df_result = pd.concat(results).fillna('desktop&mobile')
df_result

Отправляем на расчет: Desktop&Mobile 12+
Отправляем на расчет: Desktop и Mobile отдельно 12+
Отправляем на расчет: Desktop&Mobile 12-24
Отправляем на расчет: Desktop и Mobile отдельно 12-24
Отправляем на расчет: Desktop&Mobile 25-34
Отправляем на расчет: Desktop и Mobile отдельно 25-34
Отправляем на расчет: Desktop&Mobile 35-44
Отправляем на расчет: Desktop и Mobile отдельно 35-44
Отправляем на расчет: Desktop&Mobile 45-54
Отправляем на расчет: Desktop и Mobile отдельно 45-54
Отправляем на расчет: Desktop&Mobile 55+
Отправляем на расчет: Desktop и Mobile отдельно 55+
Отправляем на расчет: Desktop&Mobile Муж.12+
Отправляем на расчет: Desktop и Mobile отдельно Муж.12+
Отправляем на расчет: Desktop&Mobile Жен.12+
Отправляем на расчет: Desktop и Mobile отдельно Жен.12+
Расчет задач (16) [ = = = = = = = = = = = = = = ] время расчета: 0:00:43.756468
Расчет завершен, получаем результат
CPU times: user 826 ms, sys: 62.9 ms, total: 889 ms
Wall time: 49.1 s


,prj_name,dt_month,stat_unwreach,usetype
0,Desktop&Mobile 12+,2021-03-01,22549.0,desktop&mobile
0,Desktop и Mobile отдельно 12+,2021-03-01,19960.0,mobile-app-online
1,Desktop и Mobile отдельно 12+,2021-03-01,15607.0,mobile-app-offline
2,Desktop и Mobile отдельно 12+,2021-03-01,17953.0,mobile-web
3,Desktop и Mobile отдельно 12+,2021-03-01,18430.0,desktop
0,Desktop&Mobile 12-24,2021-03-01,2458.0,desktop&mobile
0,Desktop и Mobile отдельно 12-24,2021-03-01,2365.0,mobile-app-online
1,Desktop и Mobile отдельно 12-24,2021-03-01,1962.0,mobile-app-offline
2,Desktop и Mobile отдельно 12-24,2021-03-01,2078.0,mobile-web
3,Desktop и Mobile отдельно 12-24,2021-03-01,1752.0,desktop


In [8]:
#Выгружаем в Excel
writer = pd.ExcelWriter(rtask.get_excel_filename('UnwReach+TA_202103'))
df_result.to_excel(writer, 'Report', index=False)
writer.save()